In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://paris-olypic-2024-data@parisolympic2024avi.dfs.core.windows.net",
mount_point = "/mnt/paris_olympic",
extra_configs = configs)
  

In [0]:
%fs
ls "mnt/paris_olympic"

In [0]:
spark

In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/paris_olympic/raw-data/athletes.csv")
venues = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/paris_olympic/raw-data/venues.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/paris_olympic/raw-data/teams.csv")
medalists = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/paris_olympic/raw-data/medalists.csv")

In [0]:
df_athletes = athletes.select('name','gender','country',col('disciplines').alias('sport'),'birth_date')
df_athletes = df_athletes.withColumn('birth_date', to_date(col('birth_date')))
df_athletes = df_athletes.withColumn('sport', translate(col('sport'),"[']",''))
df_athletes.show()

+--------------------+------+--------+-----------------+----------+
|                name|gender| country|            sport|birth_date|
+--------------------+------+--------+-----------------+----------+
|    ALEKSANYAN Artur|  Male| Armenia|        Wrestling|1991-10-21|
|      AMOYAN Malkhas|  Male| Armenia|        Wrestling|1999-01-22|
|     GALSTYAN Slavik|  Male| Armenia|        Wrestling|1996-12-21|
|   HARUTYUNYAN Arsen|  Male| Armenia|        Wrestling|1999-11-22|
|     TEVANYAN Vazgen|  Male| Armenia|        Wrestling|1999-10-27|
|       ARENAS Lorena|Female|Colombia|        Athletics|      NULL|
|     McKENZIE Ashley|  Male| Jamaica|             Judo|1989-07-17|
|BASS BITTAYE Gina...|Female|  Gambia|        Athletics|      NULL|
|     CAMARA Ebrahima|  Male|  Gambia|        Athletics|1996-09-18|
| RUEDA SANTOS Lizeth|Female|  Mexico|        Triathlon|1994-03-07|
|TAPIA VIDAL Rosa ...|Female|  Mexico|        Triathlon|1997-08-27|
|   GRAJALES Crisanto|  Male|  Mexico|        Tr

In [0]:
df_venues = venues.select('venue','sports')
df_venues = df_venues.withColumn('sports', translate(col('sports'),"[']",''))
df_venues.show()

+--------------------+--------------------+
|               venue|              sports|
+--------------------+--------------------+
|     Aquatics Centre|Artistic Swimming...|
|         Bercy Arena|Artistic Gymnasti...|
|    Bordeaux Stadium|            Football|
| Champ de Mars Arena|     Judo, Wrestling|
|Château de Versai...|Equestrian, Moder...|
|Chateauroux Shoot...|            Shooting|
|Eiffel Tower Stadium|    Beach Volleyball|
|      Elancourt Hill|Cycling Mountain ...|
|Geoffroy-Guichard...|            Football|
|        Grand Palais|  Fencing, Taekwondo|
|      Hôtel de Ville|           Athletics|
|           Invalides|Archery, Athletic...|
|La Beaujoire Stadium|            Football|
|         La Concorde|3x3 Basketball, B...|
|Le Bourget Sport ...|      Sport Climbing|
|       Golf National|                Golf|
|        Lyon Stadium|            Football|
|    Marseille Marina|             Sailing|
|   Marseille Stadium|            Football|
|        Nice Stadium|          

In [0]:
df_teams = teams.select('team','discipline','events','athletes')
df_teams = df_teams.withColumnRenamed('team','country')
df_teams = df_teams.withColumnRenamed('discipline','sport')
df_teams = df_teams.withColumnRenamed('events','event')
df_teams = df_teams.withColumn('athletes', translate(col('athletes'),"[\"']/",''))
df_teams.where(col('country') == 'Brazil').show()

+-------+-------------------+--------------------+--------------------+
|country|              sport|               event|            athletes|
+-------+-------------------+--------------------+--------------------+
| Brazil|            Archery|          Mixed Team|SLIACHTICAS CAETA...|
| Brazil|          Athletics|Men's 4 x 100m Relay|BARDI Felipe, CAM...|
| Brazil|          Athletics|Men's 4 x 400m Relay|CARVALHO Lucas, H...|
| Brazil|         Basketball|                 Men|SANTOS Yago, FELI...|
| Brazil|         Equestrian|       Eventing Team|PARRO Carlos, MAM...|
| Brazil|         Equestrian|        Jumping Team|PESSOA Rodrigo, d...|
| Brazil|           Football|               Women|LORENA, ANTONIA, ...|
| Brazil|Artistic Gymnastics|        Women's Team|ANDRADE Rebeca, B...|
| Brazil|Rhythmic Gymnastics|    Group All-Around|ARAKAKI Maria Edu...|
| Brazil|           Handball|               Women|MORESCHI Gabriela...|
| Brazil|               Judo|          Mixed Team|PIMENTA Lariss

In [0]:
df_medalists = medalists.select('name','gender','medal_type','medal_code','country','discipline','event')
df_medalists = df_medalists.withColumnRenamed('name','athlete')
df_medalists = df_medalists.withColumnRenamed('medal_type','medal')
df_medalists = df_medalists.withColumnRenamed('medal_code','position')
df_medalists = df_medalists.withColumnRenamed('discipline','sport')
df_medalists.show()

+--------------------+------+------------+--------+----------------+------------+--------------------+
|             athlete|gender|       medal|position|         country|       sport|               event|
+--------------------+------+------------+--------+----------------+------------+--------------------+
|     EVENEPOEL Remco|  Male|  Gold Medal|       1|         Belgium|Cycling Road|Men's Individual ...|
|       GANNA Filippo|  Male|Silver Medal|       2|           Italy|Cycling Road|Men's Individual ...|
|       van AERT Wout|  Male|Bronze Medal|       3|         Belgium|Cycling Road|Men's Individual ...|
|         BROWN Grace|Female|  Gold Medal|       1|       Australia|Cycling Road|Women's Individua...|
|      HENDERSON Anna|Female|Silver Medal|       2|   Great Britain|Cycling Road|Women's Individua...|
|        DYGERT Chloe|Female|Bronze Medal|       3|   United States|Cycling Road|Women's Individua...|
|           OH Sanguk|  Male|  Gold Medal|       1|           Korea|     

In [0]:
## Identify country with most Gold medals
agr_medalists = df_medalists.groupBy('country','medal').agg(count('medal').alias('total_count'))
agr_medalists = agr_medalists.where(col('medal')=='Gold Medal').orderBy(col('total_count').desc())

agr_medalists.show()                            

+-------------+----------+-----------+
|      country|     medal|total_count|
+-------------+----------+-----------+
|United States|Gold Medal|        131|
|  Netherlands|Gold Medal|         66|
|        China|Gold Medal|         60|
|       France|Gold Medal|         53|
|        Spain|Gold Medal|         40|
|Great Britain|Gold Medal|         40|
|    Australia|Gold Medal|         33|
|        Italy|Gold Medal|         31|
|        Japan|Gold Medal|         27|
|  New Zealand|Gold Medal|         24|
|      Germany|Gold Medal|         22|
|        Korea|Gold Medal|         21|
|       Norway|Gold Medal|         18|
|      Denmark|Gold Medal|         17|
|       Serbia|Gold Medal|         16|
|       Canada|Gold Medal|         14|
|      Romania|Gold Medal|         12|
|      Hungary|Gold Medal|          9|
|   Uzbekistan|Gold Medal|          8|
|         NULL|Gold Medal|          6|
+-------------+----------+-----------+
only showing top 20 rows



In [0]:
df_athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/paris_olympic/transformed-data/athletes")
df_venues.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/paris_olympic/transformed-data/venues")
df_teams.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/paris_olympic/transformed-data/teams")
df_medalists.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/paris_olympic/transformed-data/medalists")